In [91]:
import requests
from datetime import datetime,timedelta
import pandas as pd

In [92]:
class Client:

    def __init__(self, API_KEY. str):

        self.API_KEY = API_KEY
        self.URL = 'https://api.currencyapi.com/v3/historical'

    def historical(self, start_date: str, end_date: str, base_currency: str = "USD") -> pd.DataFrame:
        '''Returns a DataFrame with historical currency data for a given time interval and base currency

        input:
        start_date: str, format: YYYY-MM-DD
        end_date: str, format: YYYY-MM-DD
        base_currency: str, default: USD

        output:
        pd.DataFrame
        '''
        
        df_list = []

        date_list = self.__create_time_interval(start_date, end_date)

        for date in date_list:
            data = self.__get_currency_data(date)
            df = self.__parse_currency_data(data,date)
       
            df_list.append(df)

        return pd.concat(df_list)
            
    def __create_time_interval(self, start_date:str, end_date: str) -> list:
        '''Returns a list of dates between a start and end date

        input:
        start_date: str, format: YYYY-MM-DD
        end_date: str, format: YYYY-MM-DD

        output:
        list
        '''

        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")

        date_list = []
        current_date = start
        while current_date <= end:
            date_list.append(current_date.strftime("%Y-%m-%d"))
            current_date += timedelta(days=1)

        return date_list
    
    def __get_currency_data(self, date: str) -> dict:
        '''Returns a dictionary with currency data for a given date

        input:
        date: str, format: YYYY-MM-DD

        output:
        dict
        '''

        params = {
            'date': f'{date}',
        }

        # Add your API key in the headers
        headers = {
            'apikey': self.API_KEY
        }

        # Perform the GET request
        response = requests.get(self.URL, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            return response.json()
        else:
            print(f'Failed to fetch data: {response.status_code}')

    def __parse_currency_data(self, data: str, date: str) -> pd.DataFrame:
        '''Returns a DataFrame with currency data for a given date

        input:
        data: dict
        date: str, format: YYYY-MM-DD

        '''
            
        currency_data = data['data']

        # Creating a DataFrame
        df = pd.DataFrame.from_dict(currency_data, orient='index')
        df["date"] = date
        df.rename(columns={"code": "name"}, inplace=True)

        return df
    
    def convert_to_csv(self, df, filename):
        df.to_csv(filename, index=False)



In [93]:
api_key = "cur_live_2Db4Ox8GokNGVZSFF2kk3C4qROH745UOZRuEUWDA"

In [94]:
client = Client(api_key)

In [95]:
client.convert_to_csv(client.historical('2021-01-01', '2021-01-10', 'USD'), "test.csv")

# I could not use the code snippet below because of free plan

In [23]:
"""url = "https://api.currencyapi.com/v3/range"

params = {
    "datetime_start": "2021-01-01T23:59:59Z",
    "datetime_end": "2021-01-10yT23:59:59Z",
    "accuracy": "day",
    "base_currency": "USD",
}

headers = {
    "apikey": api_key
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    # Process the response here
    data = response.json()
    print(data)
else:
    print(f"Failed to fetch data: {response}")"""


Failed to fetch data: <Response [403]>
